# LaTeX table of dynamic rupture model parameters

Generate the dynamic rupture model parameters.

2025.3.3 Kurama Okubo

- 2025.3.26 update for master table

In [ ]:
import os
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
%matplotlib inline
import numpy as np
import pandas as pd
from datetime import timedelta
from tqdm import tqdm
import warnings
import time

import scipy.integrate as integrate
import seaborn as sns 

%load_ext autoreload
%autoreload 2

plt.rcParams["font.family"] = 'Arial'
# plt.rcParams["font.sans-serif"] = "DejaVu Sans, Arial, Helvetica, Lucida Grande, Verdana, Geneva, Lucid, Avant Garde, sans-serif"
plt.rcParams["font.size"] = 11
plt.rcParams["xtick.direction"] = "in"
plt.rcParams["xtick.major.size"] = 4.75
plt.rcParams["xtick.major.width"] = 0.75
plt.rcParams["xtick.minor.size"] = 3
plt.rcParams["xtick.minor.width"] = 0.4
plt.rcParams["xtick.minor.visible"] = True

plt.rcParams["ytick.direction"] = "in"
plt.rcParams["ytick.major.size"] = 4.75
plt.rcParams["ytick.major.width"] = 0.75
plt.rcParams["ytick.minor.size"] = 3
plt.rcParams["ytick.minor.width"] = 0.4
plt.rcParams["ytick.minor.visible"] = True

plt.rcParams["savefig.transparent"] = True

plt.rcParams['axes.linewidth'] = 0.75

# Read the datasheet

In [ ]:
a_patch = 4.0e-3 # patch radius without margine
rupturetype = "pulse" # "crack": without self-healing or "pulse": with self-healing
p_dcscaleexp = 0.6 #0.55 #0.54 #0.56 #0.555  # #0.475 #0.55 #0.575 #0.65 #0.8 #0.65 #0.7

IfInitparam = True # True to make the input files with the short time duration to output the initial condition

casestudy_name = f"a={a_patch*1e3:.2f}_ruptype={rupturetype:s}_pdcscaling={p_dcscaleexp:.3f}"
print(casestudy_name)

In [ ]:
hat_sn_patch = 6e6
hat_fr_patch =0.3
stressbackground_beta = 0.35


casename = casestudy_name+"_sn={:.1f}MPa_hatfr={:.1f}_bgbeta={:.2f}".format(hat_sn_patch/1e6, hat_fr_patch, stressbackground_beta)


In [ ]:
df_dynparam = pd.read_csv(f"../data/gouge_dynamicrupture_modelparam_{casename}_master.csv", index_col=0)

In [ ]:
target_events = [24, 50, 52, 72 ,129]

In [ ]:
df_dynparam_targets = df_dynparam.loc[df_dynparam.index.isin(target_events), :].copy()
df_dynparam_targets.index = [f"M{x}" for x in df_dynparam_targets.index]

In [ ]:
df_dynparam_targets

# Compute additional values

In [ ]:
# Compute actual stress drop
df_dynparam_targets.loc[:, "$\\Delta \\tau$"] = df_dynparam_targets.apply(lambda x: x.delsig_withmargin_try * x.delsigma_factor, axis=1)

# Ds
hat_ds_factor_rapidnuc_patch = 5.5

df_dynparam_targets.loc[:, "$D_s$"] = df_dynparam_targets.apply(lambda x: x.dc_try * hat_ds_factor_rapidnuc_patch, axis=1)


# Reformat table for latex output

In [ ]:
df_dynparam_targets_selected = df_dynparam_targets[["M0_mean", "Tw_mean", "Mw_mean", "hat_sn_patch", "delsig_withmargin_try", "slip_try", "initialstress_fraction", 
                                                    "delsigma_factor", "tau_0_patch", "$\\Delta \\tau$", "fp_patch", "hat_fr", "dc_try", "$D_s$",
                                                    "GIIC_patch", "rnuc_patch"]].copy()

In [ ]:
# df_dynparam_targets_selected.index.name = "GP event ID"

## Compute the additional values

In [ ]:
df_dynparam_targets_selected

## Rename the columns

In [ ]:
df_dynparam_targets_selected.rename(
    columns={"M0_mean":"$M_0^{\\text{obs}}$", 
             "Tw_mean":"$T_w^{\\text{obs}}$",
             "Mw_mean":"$M_w^{\\text{obs}}$",
             "hat_sn_patch":"$\\sigma_n^{\\text{PHC}}$",
             "delsig_withmargin_try": "$\\Delta \\sigma_{\\text{ref}}$",
             "slip_try": "$\\bar{u}_0$",
             "initialstress_fraction": "$c$",
             "delsigma_factor": "$s$",
             "fp_patch": "$\\mu_s$",
             "hat_fr": "$\\mu_d$",
             "dc_try": "$D_c$",
             "tau_0_patch": "$\\tau_0^{\\text{PHC}}$",
             "GIIC_patch": "$G_{IIC}$",
             "rnuc_patch": "$R_{c}$",
            },
    inplace=True)

In [ ]:
df_dynparam_targets_selected

# Dump the output

## unit Convert 

In [ ]:
df_dynparam_targets_selected.columns

In [ ]:
df_dynparam_targets_selected["$T_w^{\\text{obs}}$"]


In [ ]:
s2mus = lambda x: x*1e6
Pa2MPa = lambda x: x/1e6
m2um = lambda x: x*1e6


In [ ]:
# Show GIIC with the power of 10
def GIIC_latex_conv(x):
    # ref: https://stackoverflow.com/a/59744605
    sci_notation = "{:.2e}".format(x)
    value, expscale = sci_notation.split("e")
    return "${0} \\times 10^{{{1}}}$".format(float(value), int(expscale))


In [ ]:
x = 0.021406
GIIC_latex_conv(x)

In [ ]:
# Unit conversion
formatters_targets = {"$M_0^{\\text{obs}}$": lambda x: '%.2f' % x,
                      "$T_w^{\\text{obs}}$": lambda x: '%.2f' % s2mus(x),
                      "$M_w^{\\text{obs}}$": lambda x: '%.1f' % x,
                      "$\\sigma_n^{\\text{PHC}}$": lambda x: '%.1f' % Pa2MPa(x),
                      "$\\Delta \\sigma_{\\text{ref}}$": lambda x: '%.2f' % Pa2MPa(x),
                      "$\\bar{u}_0$": lambda x: '%.2f' % m2um(x),
                      "$c$": lambda x: '%.3f' % x,
                      "$s$": lambda x: '%.3f' % x,
                      "$\\tau_0^{\\text{PHC}}$":  lambda x: '%.2f' % Pa2MPa(x),
                      "$\\Delta \\tau$":  lambda x: '%.2f' % Pa2MPa(x),
                      "$\\mu_s$": lambda x: '%.2f' % x,
                      "$\\mu_d$": lambda x: '%.1f' % x,
                      "$D_c$": lambda x: '%.3f' % m2um(x),
                      "$D_s$": lambda x: '%.3f' % m2um(x),
                      "$G_{IIC}$": lambda x: '%.2f $\\times$ 10$^{{%d}}$' % (float(f"{x:.2e}".split("e")[0]), float(f"{x:.2e}".split("e")[1])) ,
                      "$R_{c}$": lambda x: '%.2f' % x,
                     }

In [ ]:
latextable_targets = df_dynparam_targets_selected.to_latex(header=True, column_format=f"l"+"c"*(len(df_dynparam_targets_selected.columns)), formatters= formatters_targets)


In [ ]:
print(latextable_targets)

In [ ]:
df_dynparam_targets_selected